# Chapter 8. Joins

## 1. 집계
- 7장에서 본 것은 단일 데이터셋을 집계한 것
- Spark는 다양한 데이터 소스를 활용할 수 있다
- 여기서는 스파크가 클러스터에서 조인을 실행하는 방법에 대해 생각할 것
- 메모리 부족을 방지하고 이전에 해결할 수 없는 문제를 해결

### Join Expression
- *join*은 left, right 두 개의 데이터 셋을 가지고
    - 두 데이터셋에서 하나 이상의 키 값을 가지고 비교하여
    - Join expression이 올바른지 판단하여
    - Spark가 두 데이터 셋을 결합할지 판단한다
- 가장 일반적인 join 식은 equal-join으로 특정 키값이 좌우 데이터셋에서 같은지 확인한다
    - 그리고 같으면 결합한다
    - 같지 않으면 폐기한다(같은 데이터만 결합한다)
- 이 외에도 더 정교한 조인 정책이 있다
    - 복잡한 타입을 사용하거나
    - 특정 키가 있는지 체크하거나 등

### Join Types
- Join expression은 조인 여부를 결정(할지 말지)
- Join type은 결과 집합에 포함할 데이터를 결정
- Inner, outer, left outer, right outer, left semi, left anti, natural, cross
    - Exercise에서 자세히 보겠음
![join image](../../assets/presentations/week08/join_image.webp)

## 2. Challenges When Using Joins
- 조인할 때 몇 가지 이슈를 다루겠습니다
- 책의 후반부에서 다룰 최적화의 일부에 대한 스포일러를 포함합니다

### Joins on Complex Types
- 챌린징해보이지만 딱히 그래보이진 않다
- 모든 식은 Boolean을 반환한다면 유효하다

### Handling Duplicate Column Names
- Join의 까다로운 문제 중 하나는 중복된 열 이름을 처리하는 것
- 두 가지로 케이스를 나눠볼 수 있다
    - 키 칼럼에 중복이 있다
    - 키가 아닌 칼럼에 중복이 있다

## 3. How Spark Performs Joins
- 스파크가 Join을 수행하는 방법을 이해하려면 **노드 간 통신 전략**과 **노드별 계산 전략**이라는 두 가지 핵심 리소스를 이해해야 합니다
- 스파크가 조인을 수행하는 방법을 이해하는 것은 **빠르게 완료되는 작업**과 **전혀 완료되지 않는 작업**의 차이를 의미할 수 있습니다.

### Communication Strategies
- 클러스터 통신에는 두 가지 다른 방식이 있다
- 여기서 언급하는 것보다 내부적으로 훨씬 복잡한 이슈가 있다
    - 일부 최적화 중 일부는 최적화 로직이나 통신 전략의 개선으로 바뀔 수도 있다
- 그래서 여기서는 고레벨 수준의 예시를 통해 이해를 도울 것임
    - 좀 더 일반적인 시나리오를 통해, 워크로드의 속도를 높이는 낮은 성능 개선 기능을 활용할 것임
    - 빡센 건 어렵고 바뀌니까 ..
- 여기서 보는 Join의 핵심은 다음과 같다
    - 스파크에서 당신은 큰, 혹은 작은 테이블을 갖는다
    - 원래 이것(테이블의 크기)는 스펙트럼이지만
    - 두 개로 나누어 바라보는 것이 설명을 이해하는 것에 도움이 된다

**Big table-to-big table**
- 셔플 조인에서는
    - 모든 노드가 다른 노드와 대화
    - 어떤 노트에 특정 키가 있는지에 따라 데이터 공유
    - 데이터가 잘 분할되어 있지 않으면 네트워크가 혼잡해 비싸짐
- 큰 데이터 테이블에 다른 큰 데이터 테이블을 결합
    - IoT에서 매일 수십억개의 메시지를 변화를 파악하는 예시
    - 이걸 해결하기 위해 deviceId, messageType, date, yesterdayDate 칼럼 네 개를 이용한다

![big-to-big](../../assets/presentations/week08/big-to-big.png)

**Big table-to-small table**
- 하나의 테이블이 single worker 노드에 들어갈 수 있을 정도 + 여분이 좀 있을 정도로 작으면 최적화 가능
- 해당 테이블을 모두 복사떠서 모든 작업자 노드에 보내는 것
- 처음에는 대규모 통신이 필요하지만, 그 이후에 노드간 통신이 불필요해진다
- 이 경우 CPU가 가장 병목
- 코드 예시에서 explain plan을 보면 자동으로 broadcast했음을 확인 가능
![big-to-big](../../assets/presentations/week08/big-to-small.png)

**Little table-to-little table**
- 알아서 하두록 그냥 두자

## 4. Conclusion
- 언급하지 않은 것 중 하나는, 미리 데이터를 잘 분할해두면 셔플이 계획되어있더라도 스파크가 셔플을 피할 수 있다
- 일부 데이터로 파티셔닝을 미리 시도해서 join할 때 속도가 빠른지 확인해보자
- 다음 장에서는 데이터 소스 API를 설명하겠다
- 조인 순서에는 추가적인 의미가 있다
    - 일부 조인은 필터 역할을 하기 때문에, 네트워크를 통한 데이터 교환을 줄일 수 있다
- 다음 장에서는 Structured API를 이용한 데이터 읽기 및 쓰기를 볼 거임